In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import math
from math import log2
from sklearn import datasets,model_selection

In [2]:
#Creating Node here.
class TreeNode:
    def __init__(self,data,output):
        #data is the best feature upon which split was made,output is the y_prediction.
        self.data=data
        self.output=output
        #children is dictionary with key as the label of feature upon which split was made, and value is the node formed.
        self.children={}
    #Addition of child node to the parent node.
    def add_child(self,labels,child):
        self.children[labels]=child

In [3]:
class DecisionTreeClassifier:
    def __init__(self):
        self.root=None
        
        
    #This function will calculate entropy of a single node, by finding probability and multiplying it with log
    def entropy(self,y_train):
        #evaluating length of y_train
        length_y=len(y_train)
        ent=0
        #unique values present in y_train
        unique_output=set(y_train)
        for labels in unique_output:
            #will return boolean, and then .sum() will add all the true(i.e 1)
            p=(y_train==labels).sum()/length_y
            ent+=p*log2(p)
        return -1*ent
    
    
    
    def gain_ratio(self,x_train,y_train,selected_feature):
        #calculating the initial entropy of a node before split
        original_entropy=self.entropy(y_train)
        length=len(y_train)
        #selecting column with selected feature
        column_with_selected_feature=(x_train[:,selected_feature])
        # unique value that the column can take
        unique_labels=set(column_with_selected_feature)
        split_info=0
        entropy_f=0
        #now finding entropy and split ratio for all the nodes, formed after split
        for labels in unique_labels:
            #selecting only those y that has selected feature = labels
            y=y_train[column_with_selected_feature==labels]
            #applying split ratio formula and adding it.
            split_info+=(len(y)/length)*log2(len(y)/length)*-1
            #calculating weighted entropy and adding
            entropy_f+=self.entropy(y)*(len(y)/length)
        #if split_info is 0,split is not allowed,therfore return gain_ratio as -1
        if split_info==0:
            return -1
        return (original_entropy-entropy_f)/split_info

    
    
    def decision_tree(self,x_train,y_train,features,level):
        unique_y=set(y_train)
        #Pure class
        if len(unique_y)==1:
            print("level:",level)
            for i in unique_y:
                print("count of",i,":",(y_train==i).sum())
                #Finding the y_prediction,here it will be the single label that is present.
                output=i
            print("Entropy:",abs(self.entropy(y_train)))
            print("Reached Leaf Node")
            print()
            #As there is no further split, so feature selected will be None 
            return TreeNode(None,output)
        #features are over
        elif len(features)==0:
            max_freq=0
            print("level:",level)
            for i in unique_y:
                print("count of",i,":",(y_train==i).sum())
                #Finding the class with majority, as that will be the y_prediction
                if max_freq<(y_train==i).sum():
                    output=i
                    max_freq=(y_train==i).sum()
            print("Entropy:",abs(self.entropy(y_train)))
            print("Reached Leaf Node")
            print()
            #As there is no further split, so feature selected will be None 
            return TreeNode(None,output)
        else:
            #finding out the best split
            max_gain=0
            selected_feature=None
            #splitting on every feature to decide best gain_ratio
            for f in features:
                gain=self.gain_ratio(x_train,y_train,f)
                if(max_gain<gain):
                    max_gain=gain
                    selected_feature=f
            #max_gain will be 0 in case split info is 0,or information gain is 0,in both this case there will be no split
            if max_gain==0:
                return TreeNode(None,None)
            print("level:",level)
            max_freq=0
            for i in unique_y:
                print("count of",i,":",(y_train==i).sum())
                if max_freq<(y_train==i).sum():
                    output=i
                    max_freq=(y_train==i).sum()
            #Made a node,data is here selected_feature and output is what we found above.
            current_Node=TreeNode(selected_feature,output)
            inde=features.index(selected_feature)
            #removing the selected feature from list
            features.remove(selected_feature)
            column_with_selected_feature=x_train[:,selected_feature]
            unique_labels_in_selected_feature=set(column_with_selected_feature)
            print("Entropy:",abs(self.entropy(y_train)))
            print("Splitting on feature",selected_feature,"with gain-ratio",max_gain)
            print()
            #now will call recursion for all whose values which the selected feature can take
            for labels in unique_labels_in_selected_feature:

                x_new=x_train[column_with_selected_feature==labels]
                y_new=y_train[column_with_selected_feature==labels]
                child_node=self.decision_tree(x_new,y_new,features,level+1)
                #adding the child_node to the current node.
                current_Node.add_child(labels,child_node)
            #Adding the removed feature again
            features.insert(inde,selected_feature)
            return current_Node
        
        
        
        
    #converting continuous data to discrete data by mean method
    def labelled_data(self,x_train):
        mean=x_train.mean()
        first_mean=0.5*mean
        second_mean=1.5*mean
        for i in range(len(x_train)):
            if x_train[i]<first_mean:
                x_train[i]=0
            elif x_train[i]<mean:
                x_train[i]=1
            elif x_train[i]<second_mean:
                x_train[i]=2
            else:
                x_train[i]=3
        return x_train
    
    
    
    
    def fit(self,x_train,y_train):
        #converting continuous data into discrete,column wise
        for col in range(x_train.shape[1]):
            x_train[:,col]=self.labelled_data(x_train[:,col])
        features=[i for i in range(x_train.shape[1])]
        #storing the root, and will access rest as done in TREE
        self.root=self.decision_tree(x_train,y_train,features,0)
        
        
    
    def predict_for_single(self,X,node):
        #will reach leaf node and predict output
        if len(node.children)==0:
            return node.output
        val=X[node.data]
        #if the label is not present in children, there will be no further split and declare the output there itself.
        if val not in node.children:
            return node.output
        #recursively finding output
        return self.predict_for_single(X,node.children[val])
    
    def predict(self,X):
        length=X.shape[0]
        y_pred=[]
        #will predict output for each testing point and will append it in y_pred
        for i in range(length):
            y=self.predict_for_single(X[i,:],self.root)
            y_pred.append(y)
        return y_pred
    
    
    def score(self,X,Y):
        Y_pred=self.predict(X)
        #will just find correct y divide by total y
        correct_output=(Y_pred==Y).sum()
        return correct_output/len(Y)

In [4]:
iris=datasets.load_iris()
x = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]])

y = np.array([0,
              1,
              1,
              1])
clf=DecisionTreeClassifier()
clf.fit(x,y)
y_pred=clf.predict(x)
print("score:",clf.score(x,y))

level: 0
count of 0 : 1
count of 1 : 3
Entropy: 0.8112781244591328
Splitting on feature 0 with gain-ratio 0.31127812445913283

level: 1
count of 0 : 1
count of 1 : 1
Entropy: 1.0
Splitting on feature 1 with gain-ratio 1.0

level: 2
count of 0 : 1
Entropy: 0.0
Reached Leaf Node

level: 2
count of 1 : 1
Entropy: 0.0
Reached Leaf Node

level: 1
count of 1 : 2
Entropy: 0.0
Reached Leaf Node

score: 1.0


In [5]:
#Trying on iris dataset
iris=datasets.load_iris()
X=iris.data
Y=iris.target
clf=DecisionTreeClassifier()
clf.fit(X,Y)
y_pred=clf.predict(X)
print("score:",clf.score(X,Y))
#Score is not 1 becuase we have done by converting continuous into categorical

level: 0
count of 0 : 50
count of 1 : 50
count of 2 : 50
Entropy: 1.584962500721156
Splitting on feature 3 with gain-ratio 0.7350016280496156

level: 1
count of 0 : 49
Entropy: 0.0
Reached Leaf Node

level: 1
count of 0 : 1
count of 1 : 10
Entropy: 0.4394969869215134
Splitting on feature 1 with gain-ratio 1.0

level: 2
count of 1 : 10
Entropy: 0.0
Reached Leaf Node

level: 2
count of 0 : 1
Entropy: 0.0
Reached Leaf Node

level: 1
count of 1 : 39
count of 2 : 5
Entropy: 0.5107878229540133
Splitting on feature 2 with gain-ratio 0.2488471906913506

level: 2
count of 1 : 1
Entropy: 0.0
Reached Leaf Node

level: 2
count of 1 : 38
count of 2 : 4
Entropy: 0.4537163391869448
Splitting on feature 1 with gain-ratio 0.04070432026142338

level: 3
count of 1 : 31
count of 2 : 4
Entropy: 0.512709142030877
Splitting on feature 0 with gain-ratio 0.012981006561098145

level: 4
count of 1 : 14
count of 2 : 1
Entropy: 0.35335933502142136
Reached Leaf Node

level: 4
count of 1 : 17
count of 2 : 3
Entropy: